# Validate Minian outputs

- Load
    - spatial map (A) -> maybe to associate with a projection of neuron fluorescence
    - temporal activity of detected neurons

- Validate units
- Extract and save relevant data for each selected unit
    - spatial location (x, y); no need for shape
    - Ca2+ trace
    - Ca2+ peak time and amplitude
    - Deconvolved spikes


Load the required packages (and many more that are not useful)

In [ ]:

import numpy as np
import csv
from pathlib import Path
import xarray as xr
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
import itertools as itt
import os
import sys
import json
from scipy import signal
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

import holoviews as hv
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from holoviews import opts
from holoviews.operation.datashader import shade
hv.extension('bokeh', 'matplotlib')
from IPython.core.display import display

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)


#%reset

Import the minian files

In [ ]:
folderGen = Path('.').absolute()
recording = 'Baseline_recording'
experimenter = 'Gaelle'
mouse_name = 'BlueLines'
date = '2022_08_06_13_46_59'
device = 'V4_Miniscope'
#time = '10_01_23'
minianversion = 'minian'

folderMouse = Path(f'{folderGen}/{experimenter}/{recording}/{mouse_name}/{date}/{device}/{minianversion}/')

minian_ds = open_minian(folderMouse)

Import spatial map, Ca2+ traces, deconvolved spikes

In [ ]:
A = minian_ds['A']
C = minian_ds['C']
S = minian_ds['S']

B = A['unit_id']
series = B.to_series()
D = series.count()

idloc = A.idxmax("unit_id")
Hmax = A.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = A.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a = pd.concat([coord1,coord2], axis=1)
unit = len(a)


Plot either all cells or just the one of interest

Plot the spatial map for all cells + interactive Ca2+ trace

In [ ]:

output_size = 150
hv.output(size=int(output_size))
opts = dict(
    plot=dict(colorbar=True, invert_yaxis=True),
    style=dict(cmap="Viridis"),
)
image = hv.Image(
    A.max("unit_id").compute().astype(np.float32).rename("A"),
    kdims=["width", "height"],
).opts(**opts)
labels = hv.Labels([(a.iloc[i,0], a.iloc[i,1], a.index[i]) for i in range(unit)])
plot_unit = hv.HoloMap({i: hv.Curve(((C[i,:])), group='Group', label=f'{a.index[i]}') for i in range(unit)}, 'Value').opts(ylim=(-0.5, 20))
labels2 = hv.Labels((150, 150, a.index[1]))
layout = image * labels + plot_unit * labels2
layout

Remove dropped units

In [ ]:
# fill in unit_to_drop with cell_id of cells to drop
unit_to_drop = [197]
copyB = list(B.copy())
for i in range(len(unit_to_drop)):
    elem = unit_to_drop[i]
    copyB.remove(elem)
unit_to_keep = copyB

A_upd = A.loc[unit_to_keep,:,:]
C_upd = C.loc[unit_to_keep,:]
S_upd = S.loc[unit_to_keep,:]

TodropFile = folderMouse / f'TodropFile.json'

with open(TodropFile, 'w') as f:
    json.dump(unit_to_drop, f, indent=2) 

Extract Ca2+ peaks

In [ ]:
## For one neuron, need to be implemented for the whole set and concatenate the np into one xarray with 

Indiv_trace = C_upd[2,:].to_series()
peaks, properties = find_peaks(Indiv_trace)
# peak boundaries taken at 70% from peak of intensity. This means that the peaks with small amplitude will be longer than the big ones.
results_width = peak_widths(Indiv_trace, peaks, rel_height=0.7)
# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
peak_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()


In [ ]:
for i in range(len(unit_to_keep)):
    Indiv_trace = C_upd[i,:].to_series()
    peaks, properties = find_peaks(Indiv_trace)
# peak boundaries taken at 70% from peak of intensity. This means that the peaks with small amplitude will be longer than the big ones.
    results_width = peak_widths(Indiv_trace, peaks, rel_height=0.7)
# Organise results in numpy array
    peaks2 = peaks.reshape(len(peaks),1)
    npresults_width = np.array(results_width).reshape(4,-1)
    peak_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()
    


In [ ]:
for i in range(len(unit)):
    print(i)
    subfolder = file_path.parents[1].stem
    if subfolder == 'continuous':
        recording = file_path.parents[2].stem.replace('recording','')
        print(recording)
        file = file_path.stem
        print(recording, file)
        np_arr = np.load(file_path)
        datalen = len(np_arr)
        print(file, datalen)
        if recording == 1: #not in TTL_stamp2:
            TTL_stamp2.append(recording)
            coords = {
                'channels' : np.array(['synchronized_timestamps', 'timestamps']),
                'duration_rec' : np.arange(datalen)
            }
            globals()[f"StampsCont_{recording}"] = xr.DataArray(coords=coords, dims=['channels', 'duration_rec'])
        globals()[f"StampsCont_{recording}"].loc[file,:] = np_arr   


In [ ]:
score=[1,2,3,4,5]
TodropFile = folderMouse / f'TodropFile.json'

with open(TodropFile, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(score, f, indent=2) 

with open(TodropFile, 'r') as f:
    score = json.load(f)

print(score)


# fill in unit_to_drop with cell_id of cells to drop
# RedLines Rec1 unit_to_drop = [0, 1, 6, 15, 18, 20, 21, 24, 26, 27, 28]
# RedLines Rec3 unit_to_drop = [10, 12, 13, 17]
# RedLines Rec2 unit_to_drop = [13, 14, 15]
# GreenDots Rec2 unit_to_drop = [0, 2, 16, 30, 56] # 30 is an unit but the second part of the record (from 25000 there must have been movent and few "cells" show similar signal)
# GreenLines Rec2 unit_to_drop = [2]
# ThreeBlueCross Rec1 unit_to_drop = [30]